In [33]:
Position = tuple[int, int]
HistoryElement = tuple[Position, str]


def get_next_position(position: Position, direction: str) -> Position:
    x, y = position

    if direction == "R":
        return (x + 1, y)
    elif direction == "L":
        return (x - 1, y)
    elif direction == "U":
        return (x, y - 1)
    elif direction == "D":
        return (x, y + 1)

    raise Exception("Invalid direction")


def get_next_direction(mirror: str, direction: str) -> str:
    if mirror == "\\":
        if direction == "R":
            return "D"
        elif direction == "L":
            return "U"
        elif direction == "U":
            return "L"
        elif direction == "D":
            return "R"

    if mirror == "/":
        if direction == "R":
            return "U"
        elif direction == "L":
            return "D"
        elif direction == "U":
            return "R"
        elif direction == "D":
            return "L"

    raise Exception("Invalid direction or mirror")


def iterate(
    input_string: str,
    position: Position = (0, 0),
    direction: str = "R",
    previous: set[HistoryElement] = set(),
) -> set[HistoryElement]:
    map = input_string.splitlines()
    x, y = position

    if (position, direction) in previous:
        return previous

    if y < 0 or x < 0 or y >= len(map) or x >= len(map[y]):
        return previous

    operator = map[y][x]

    if (
        operator == "."
        or (operator == "-" and direction in "LR")
        or (operator == "|" and direction in "UD")
    ):
        return iterate(
            input_string,
            get_next_position(position, direction),
            direction,
            previous.union(set([(position, direction)])),
        )

    if operator in "\\/":
        next_direction = get_next_direction(operator, direction)
        return iterate(
            input_string,
            get_next_position(position, next_direction),
            next_direction,
            previous.union(set([(position, direction)])),
        )

    if operator == "|" and direction in "LR":
        up_history = iterate(
            input_string,
            get_next_position(position, "U"),
            "U",
            previous.union(set([(position, "L"), (position, "R")])),
        )

        return iterate(
            input_string,
            get_next_position(position, "D"),
            "D",
            up_history.copy(),
        )

    if operator == "-" and direction in "UD":
        left_history = iterate(
            input_string,
            get_next_position(position, "L"),
            "L",
            previous.union(set([(position, "U"), (position, "D")])),
        )

        return iterate(
            input_string,
            get_next_position(position, "R"),
            "R",
            left_history.copy(),
        )

    raise Exception("Invalid map")


def get_energy(history: set[HistoryElement]) -> int:
    return len(list(set([i for i, _ in history])))

In [34]:
test_input = """\
.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\\
..../.\\\\..
.-.-/..|..
.|....-|.\\
..//.|...."""

assert get_energy(iterate(test_input)) == 46

In [35]:
value = get_energy(iterate(open("16.txt").read()))

print(f"Part 1: {value}")

assert value == 6795

RecursionError: maximum recursion depth exceeded

In [ ]:
todos = []
values = []
game_map = test_input.splitlines()

for y in range(len(game_map)):
    todos.append(((0, y), "R"))
    todos.append(((len(game_map[y]) - 1, y), "L"))

for x in range(len(game_map[0])):
    todos.append(((x, 0), "D"))
    todos.append(((x, len(game_map) - 1), "U"))

assert max([get_energy(iterate(test_input, pos, dir)) for pos, dir in todos]) == 51

In [ ]:
todos = []
values = []
input_string = open("16.txt").read()
game_map = input_string.splitlines()

for y in range(len(game_map)):
    todos.append(((0, y), "R"))
    todos.append(((len(game_map[y]) - 1, y), "L"))

for x in range(len(game_map[0])):
    todos.append(((x, 0), "D"))
    todos.append(((x, len(game_map) - 1), "U"))


def f(pos, dir):
    try:
        return get_energy(iterate(input_string, pos, dir))
    except:
        return 0


print(f"Part 2: {max([f(pos, dir) for pos, dir in todos])}")